In [ ]:
import scipy.linalg as spl
import jax.numpy as jnp
from scipy.integrate import odeint

# Beam paremeters for 2 mode system
k_nl = 4250000
w_1 = 91.734505484821950
w_2 = 3.066194429903638e02
zeta_1 = 0.0
zeta_2 = 0.0
phi_L = jnp.array([[-7.382136522799137, 7.360826867549465]])

# Modal Matrices
M = jnp.eye(2)
C = jnp.array([[2 * zeta_1 * w_1, 0], [0, 2 * zeta_2 * w_2]])
K = jnp.array([[w_1**2, 0], [0, w_2**2]])
Minv = spl.inv(M)

def model_ode_anal(t, X, T, F):
    # State equation of the mass spring system. Xdot(t) = g(X(t))
    x = X[:2]
    xdot = X[2:]
    KX = K @ x
    CXdot = C @ xdot
    force = jnp.array([F * jnp.sin(2 * jnp.pi / T * t), 0])
    phi_x = phi_L @ x  # Physical displacement at nonlinear location
    fnl = k_nl * phi_L.T @ (phi_x**3)
    Xdot = jnp.concatenate((xdot, Minv @ (-KX - CXdot - fnl + force)))
    return Xdot

def periodicity_anal(X0, T, F):
    t = jnp.linspace(0, T, 10 + 1)
    Xsol = odeint(model_ode_anal, X0, t, args=(T, F), rtol=1e-8, tfirst=True)
    H = Xsol[-1, :] - Xsol[0, :]
    return H.reshape(-1, 1)

X0 = jnp.array([0.0, 0.0, 1.0, 1.0])
T = 2.0
F = 10.0

periodicity_anal(X0, T, F)

/tmp/ipykernel_1618482/2511633045.py:45: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  Xsol = odeint(model_ode_anal, X0, t, args=(T, F), rtol=1e-8, tfirst=True)


array([[ 0.36862745],
       [ 0.30980392],
       [-0.36470588],
       [ 0.        ]])

In [30]:
import scipy.linalg as spl
import jax.numpy as jnp
from scipy.integrate import solve_ivp

# Beam paremeters for 2 mode system
k_nl = 4250000
w_1 = 91.734505484821950
w_2 = 3.066194429903638e02
zeta_1 = 0.0
zeta_2 = 0.0
phi_L = jnp.array([[-7.382136522799137, 7.360826867549465]])

# Modal Matrices
M = jnp.eye(2)
C = jnp.array([[2 * zeta_1 * w_1, 0], [0, 2 * zeta_2 * w_2]])
K = jnp.array([[w_1**2, 0], [0, w_2**2]])
Minv = spl.inv(M)

def model_ode_anal(t, X, T, F):
    # State equation of the mass spring system. Xdot(t) = g(X(t))
    x = X[:2]
    xdot = X[2:]
    KX = K @ x
    CXdot = C @ xdot
    force = jnp.array([F * jnp.sin(2 * jnp.pi / T * t), 0])
    phi_x = phi_L @ x  # Physical displacement at nonlinear location
    fnl = k_nl * phi_L.T @ (phi_x**3)
    Xdot = jnp.concatenate((xdot, Minv @ (-KX - CXdot - fnl + force)))
    return Xdot

def periodicity_anal(X0, T, F):
    t = jnp.linspace(0, T, 10 + 1)
    Xsol = solve_ivp(model_ode_anal, (t[0], t[-1]), X0, args=(T, F), t_eval=jnp.linspace(0, 2, 10 + 1), rtol=1e-8)
    H = Xsol.y[:, -1] - Xsol.y[:, 0]
    return H.reshape(-1, 1)

X0 = jnp.array([0.0, 0.0, 1.0, 1.0])
T = 2.0
F = 10.0

periodicity_anal(X0, T, F)

array([[-0.00490432],
       [-0.00292757],
       [-1.94054514],
       [-1.08723555]])

In [ ]:
import scipy.linalg as spl
import jax.numpy as jnp
import jax.experimental.ode as jode

# Beam paremeters for 2 mode system
k_nl = 4250000
w_1 = 91.734505484821950
w_2 = 3.066194429903638e02
zeta_1 = 0.0
zeta_2 = 0.0
phi_L = jnp.array([[-7.382136522799137, 7.360826867549465]])

# Modal Matrices
M = jnp.eye(2)
C = jnp.array([[2 * zeta_1 * w_1, 0], [0, 2 * zeta_2 * w_2]])
K = jnp.array([[w_1**2, 0], [0, w_2**2]])
Minv = spl.inv(M)

def model_ode_anal(X, t, T, F):
    # State equation of the mass spring system. Xdot(t) = g(X(t))
    x = X[:2]
    xdot = X[2:]
    KX = K @ x
    CXdot = C @ xdot
    force = jnp.array([F * jnp.sin(2 * jnp.pi / T * t), 0])
    phi_x = phi_L @ x  # Physical displacement at nonlinear location
    fnl = k_nl * phi_L.T @ (phi_x**3)
    Xdot = jnp.concatenate((xdot, Minv @ (-KX - CXdot - fnl + force)))
    return Xdot

def periodicity_anal(X0, T, F):
    t = jnp.linspace(0, T, 10 + 1)
    Xsol = jode.odeint(model_ode_anal, X0, t, T, F, rtol=1e-8)

    H = Xsol[-1, :] - Xsol[0, :]
    return H.reshape(-1, 1)

X0 = jnp.array([0.0, 0.0, 1.0, 1.0])
T = 2.0
F = 10.0

periodicity_anal(X0, T, F)

Array([[-0.00528779],
       [-0.00311958],
       [-1.8358393 ],
       [-0.9015873 ]], dtype=float32)

In [ ]:
import scipy.linalg as spl
import jax
import jax.numpy as jnp
from diffrax import diffeqsolve, ODETerm, Tsit5, SaveAt, PIDController

# Beam paremeters for 2 mode system
k_nl = 4250000
w_1 = 91.734505484821950
w_2 = 3.066194429903638e02
zeta_1 = 0.0
zeta_2 = 0.0
phi_L = jnp.array([[-7.382136522799137, 7.360826867549465]])

# Modal Matrices
M = jnp.eye(2)
C = jnp.array([[2 * zeta_1 * w_1, 0], [0, 2 * zeta_2 * w_2]])
K = jnp.array([[w_1**2, 0], [0, w_2**2]])
Minv = spl.inv(M)

def model_ode_anal(t, X, args):
    T, F = args
    # State equation of the mass spring system. Xdot(t) = g(X(t))
    x = X[:2]
    xdot = X[2:]
    KX = K @ x
    CXdot = C @ xdot
    force = jnp.array([F * jnp.sin(2 * jnp.pi / T * t), 0])
    phi_x = phi_L @ x  # Physical displacement at nonlinear location
    fnl = k_nl * phi_L.T @ (phi_x**3)
    Xdot = jnp.concatenate((xdot, Minv @ (-KX - CXdot - fnl + force)))
    return Xdot

term=ODETerm(model_ode_anal)
solver=Tsit5()
saveat = SaveAt(ts=jnp.linspace(0, 2, 10 + 1))

def periodicity_anal(X0, T, F):
    t = jnp.linspace(0, T, 10 + 1)
    Xsol = diffeqsolve(term, solver, t0=t[0], t1=t[-1], dt0=0.2, y0=X0, args=(T, F), saveat=saveat, stepsize_controller=PIDController(rtol=1e-8, atol=1e-8),max_steps=None)

    H = Xsol.ys[-1, :] - Xsol.ys[0, :]
    return H.reshape(-1, 1)

X0 = jnp.array([0.0, 0.0, 1.0, 1.0])
T = 2.0
F = 10.0

periodicity_anal(X0, T, F)

Array([[-0.00473348],
       [-0.0035218 ],
       [-1.7558949 ],
       [-1.2415923 ]], dtype=float32)